# 第3节：校准与基准稳态 

## 表 3.1--模型核心参数校准

In [5]:
% =========================================================================
% == SCRIPT: Tab3_1.m
% == 版本: [v2.7 - 精简换行版]
% ==
% == 核心修改:
% ==   - [!!! 格式优化 !!!] 采纳您的建议，在 \midrule, \toprule,
% ==     \bottomrule 和 \multicolumn 这些本身就定义了行结构的命令后，
% ==     不再输出多余的换行符 "\\"。
% =========================================================================
clear; clc;
addpath(pwd);
fprintf('--- 启动 LaTeX 参数表直接生成脚本 (v2.7) ---\n');

%% --- 1. 加载数据并定义输出文件名 ---
data_filename = 'SS/data_for_transition.mat';
output_filename = 'tex/tab/Tab3.1.tex';

try
    load(data_filename, 'data_for_transition');
    cS = data_for_transition.cS;
    fprintf('   ✅ 已从 "%s" 加载参数结构体 cS。\n', data_filename);
catch ME
    error('无法加载转轨数据文件 "%s"。请先成功运行 main_run_ss.m。\n错误信息: %s', data_filename, ME.message);
end

%% --- 2. 构建参数数据 (Cell Array) ---
% 格式: { '命令或参数名', '符号', '数值', '来源', is_full_line_command }
% 最后一列是布尔值，用于标记是否为独占一行的命令
params_data = {
    '\midrule[0.75pt]', '', '', '', true; 
    '\multicolumn{4}{l}{\textbf{A. 偏好与生命周期参数}}\\', '', '', '', true;
    '相对风险规避系数', '$\sigma$', sprintf('%.1f', cS.sigma), '标准宏观文献', false;
    '主观贴现因子 (年化)', '$\beta_{annual}$', sprintf('%.3f', cS.beta^(1/cS.time_Step)), '校准目标', false;
    '人口稳态增长率 (年化)', '$n_{ss}$', sprintf('%.1f%%', cS.n_ss * 100), '长期人口预测', false;
    '存活率', '$\phi_a$', '时变', 'UN WPP (2024)', false;
    '模型期长度', '', sprintf('%d年', cS.time_Step), '模型设定', false;
    '退休年龄', '$a_{R}$', sprintf('%d岁', cS.ageRetire_orig), '模型设定', false;
    '\midrule', '', '', '', true; 
    '\multicolumn{4}{l}{\textbf{B. 生产技术参数}}\\', '', '', '', true;
    '技术进步率 (年化)', '$g_{A,ss}$', sprintf('%.1f%%', cS.g_A_ss * 100), '长期增长假设', false;
    '私人资本产出弹性', '$\alpha$', sprintf('%.3f', cS.alpha), 'Penn World Table', false;
    '公共资本产出弹性', '$\gamma$', sprintf('%.3f', cS.gamma), '校准目标', false;
    '私人资本折旧率 (年化)', '$\delta_k$', sprintf('%.1f%%', (1-(1-cS.ddk)^(1/cS.time_Step))*100), 'NIPA数据', false;
    '公共资本折旧率 (年化)', '$\delta_g$', sprintf('%.1f%%', (1-(1-cS.ddk_g)^(1/cS.time_Step))*100), '校准目标', false;
    '\midrule', '', '', '', true;
    '\multicolumn{4}{l}{\textbf{C. 财政与PAYG养老金}}\\', '', '', '', true;
    '资本所得税率', '$\tau_k$', sprintf('%.1f%%', cS.tau_k * 100), '中国现行税制', false;
    '劳动所得税率', '$\tau_l$', sprintf('%.1f%%', cS.tau_l * 100), '中国现行税制', false;
    '消费税率', '$\tau_c$', sprintf('%.1f%%', cS.tau_c * 100), '中国现行税制', false;
    '政府消费 / GDP', '$G_c/Y$', sprintf('%.1f%%', cS.G_c_to_Y_ratio_ss * 100), '财政数据', false;
    '公共投资 / GDP', '$I_g/Y$', sprintf('%.1f%%', cS.I_g_to_Y_ratio_ss * 100), '财政数据', false;
    'PAYG缴费率', '$\theta_t$', '时变路径', '内生以平衡PAYG', false;
    '\midrule', '', '', '', true;
    '\multicolumn{4}{l}{\textbf{D. 个人养老金 (PPS) 制度}}\\', '', '', '', true;
    'PPS最大缴费率 (占劳动收入)', '$pps_{max}$', sprintf('%.1f%%', cS.pps_max * 100), '中国个人养老金政策', false;
    'PPS提取率 (占PPS资产)', '', sprintf('%.1f%%', cS.pps_withdrawal_rate * 100), '模型简化假设', false;
    'PPS提取税率', '$\tau_{pps}$', sprintf('%.1f%%', cS.pps_tax_rate_withdrawal * 100), '中国个人养老金政策', false;
};
fprintf('   ✅ 已构建参数数据。\n');

%% --- 3. 准备输出目录并打开文件 ---
[output_dir, ~, ~] = fileparts(output_filename);
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('   已创建新目录: "%s"\n', output_dir);
end

fileID = fopen(output_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开最终输出文件: %s', output_filename); end
fprintf('   正在将完整的LaTeX表格代码写入 "%s"...\n', output_filename);

% --- 写入表格外层框架 ---
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{模型核心参数校准}\n');
fprintf(fileID, '\\label{tab:param_calibration}\n');
fprintf(fileID, '\\begin{threeparttable}\n');
fprintf(fileID, '\\resizebox{\\textwidth}{!}{\n');
fprintf(fileID, '\\begin{tabular}{lccc}\n');
fprintf(fileID, '\\toprule\n');
fprintf(fileID, '参数名 & 参数符号 & 参数值 & 参数来源 \\\\\n');

% --- 循环写入表格主体内容 ---
for i = 1:size(params_data, 1)
    % --- 统一进行转义处理 ---
    col1 = strrep(params_data{i, 1}, '%', '\%');
    col2 = strrep(params_data{i, 2}, '%', '\%');
    col3 = strrep(params_data{i, 3}, '%', '\%');
    col4 = strrep(params_data{i, 4}, '%', '\%');
    is_command = params_data{i, 5};
    
    if is_command
        % 如果是整行命令，直接打印，不加换行符 "\\"
        fprintf(fileID, '%s\n', col1);
    else
        % 如果是普通的数据行，用 "&" 连接并换行
        fprintf(fileID, '%s & %s & %s & %s \\\\\n', col1, col2, col3, col4);
    end
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}}\n'); 
fprintf(fileID, '\\begin{tablenotes}[para]\n');
fprintf(fileID, '  \\item[注] 主观贴现因子、折旧率和人口增长率已根据模型周期长度 (%d年) 进行年化处理以便解读。\n', cS.time_Step);
fprintf(fileID, '  PAYG缴费率($\\theta_t$)和存活率($\\phi_a$)为时变路径，详情见正文。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}');

% --- 关闭文件 ---
fclose(fileID);

fprintf('   ✅ 成功生成语法正确的LaTeX表格文件: %s\n', output_filename);
fprintf('\n--- 脚本执行完毕 ---\n');

--- 启动 LaTeX 参数表生成脚本 ---
   ✅ 已从 "SS/data_for_transition.mat" 加载参数结构体 cS。
   ✅ 已根据模型设定构建参数表。
   ✅ 已调用 table2latex 生成临时文件 "temp_table_for_script.tex"。
   正在将表格内容写入最终框架并保存至 "Tab3.1.txt"...
   ✅ 成功生成最终的LaTeX表格文件: Tab3.1.txt

--- 脚本执行完毕 ---


# 第4节

## 图 4.1--人口冲击压力下的宏观经济转轨动态 ($\bar{g}=2\%$)

In [ ]:
% =========================================================================
% == SCRIPT: plot_figure_4_1_aging_impact_dashboard_v4.2.m
% ==
% == 版本: [v4.2 - 1x4核心变量最终版]
% ==
% == 目的:
% ==   - [核心] 最终精简版，聚焦于4个最重要的宏观变量：人均资本(k_p),
% ==     人均产出(y), 年化利率(r), 和有效工资率(w_hat)。
% ==   - [布局] 采用1x4的单行布局，使图表更紧凑，适合在论文中横向放置。
% ==   - [美学] 保持所有精修指令：移除轴标签、信息融入标题、单一图例、
% ==     固定Y轴、调整图窗尺寸、无总标题。
% ==   - [功能] 仍然保留 year_max 参数，灵活控制图表展示的时间范围。
% ==
% == 前置条件:
% ==   - TRANS/TPI_results_main_nopps.mat
% =========================================================================

clear; close all;

fprintf('--- 启动图4.1绘制脚本 (v4.2 - 1x4核心变量最终版) ---\n');

%% --- 1. 用户设定 ---
year_max = 2300; % 控制图表展示的结束年份

%% --- 2. 定义文件路径与检查 ---
file_nopps = 'TRANS/TPI_results_main_nopps.mat';
output_dir = 'tex/fig';
output_filename = fullfile(output_dir, 'fig4.1_aging_impact_dashboard.png');

if ~exist(file_nopps, 'file'), error('找不到PAYG-only结果文件: %s', file_nopps); end
if ~exist(output_dir, 'dir'), mkdir(output_dir); fprintf('   已创建输出目录: %s\n', output_dir); end

%% --- 3. [核心] 数据加载与重构 ---
fprintf('   正在加载并重构 "PAYG-only" 情景数据...\n');

% --- 数据加载 ---
data = load(file_nopps);
cS = data.cS;

% --- 路径重构 ---
total_pop_path = sum(cS.Z_path_raw, 1);
AZ_path = cS.A_path .* total_pop_path;

paths = struct();
paths.k_p = data.results.K_p_path ./ AZ_path;
paths.y   = data.results.Y_path   ./ AZ_path;
paths.r   = data.results.r_path; 
paths.w_hat = data.results.w_path ./ cS.A_path;

% 计算退休人口占比
mass_retirees_path = sum(cS.Z_path_raw((cS.aR_new+1):end, :), 1);
paths.retiree_share = mass_retirees_path ./ total_pop_path;

% --- 时间轴截取 ---
T = cS.T_sim;
time_axis_full = (cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step));

% [!!! 核心修正: 重新定义 xlabel_text !!!]
xlabel_text = '年份'; % 确保该变量被定义

time_mask = time_axis_full <= year_max;
time_axis = time_axis_full(time_mask);

% 对所有路径应用mask
fields = fieldnames(paths);
for i = 1:length(fields)
    if size(paths.(fields{i}), 2) == T
        paths.(fields{i}) = paths.(fields{i})(time_mask);
    end
end

fprintf('   ✅ 数据加载和重构完毕。\n');

%% --- 4. 绘图 ---
fprintf('   正在生成图表...\n');
fig = figure('Name', '老龄化冲击下的宏观经济动态', 'Position', [461 603 730 319]); % 调整高度以适应1x2布局
tcl = tiledlayout(1, 2, 'Padding', 'compact', 'TileSpacing', 'normal');

% =========================================================================
% ==                      [核心美学修正 - 1x2 复合版]
% =========================================================================
% --- 1. 定义灰度图风格 ---
style_kp   = {'k-', 'LineWidth', 2.5};     % 资本: 黑色实线
style_y    = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2.5}; % 产出: 深灰虚线
style_w    = {':', 'Color', [0.2 0.2 0.2], 'LineWidth', 2.5}; % 工资: 黑色点线
style_r    = {'k-', 'LineWidth', 2.5};     % 利率: 黑色实线 (在自己的图里)
style_bg_line  = {'-.', 'Color', [0.8 0.8 0.8], 'LineWidth', 2}; % 背景线: 浅灰色虚线

color_left_axis = 'black';
color_right_axis = [0.5 0.5 0.5]; % 右轴: 中灰色

font_size_title = 14;
font_size_axis  = 11;
yy_lim_bar = [15 60]; % 固定Y轴范围 (百分比)

% --- 2. 准备X轴刻度 (确保包含首尾年份) ---
tick_start_year = ceil(time_axis(1)/50)*50;
xticks_vec = tick_start_year :30 : time_axis(end);
% 确保第一个和最后一个年份刻度被包括
xticks_vec = unique([time_axis(1), xticks_vec]);


% --- 面板 1: 资本、产出与工资 ---
ax1 = nexttile;
hold(ax1, 'on');

% --- 设置坐标轴本身的字体为 Times New Roman ---
% 这会影响到刻度标签（数字和英文）
set(ax1, 'FontName', 'Times New Roman');
set(ax1, 'FontSize', font_size_axis); % 保持轴字体大小


% -- 右轴: 背景线 --
yyaxis(ax1, 'right');
p_bg_line = plot(ax1, time_axis, paths.retiree_share * 100, style_bg_line{:},'DisplayName',['退休人口占比' newline '(右轴, %)']);
set(ax1, 'YColor', color_right_axis);
ylim(ax1, yy_lim_bar);
% 由于已经设置了ax1的FontName，Y轴刻度会自动继承Times New Roman。
% 如果有YLabel，显式设置其字体为SimSun
ylabel_right_ax1 = get(ax1, 'YLabel');
if ~isempty(ylabel_right_ax1)
    set(ylabel_right_ax1, 'FontName', 'SimSun');
end


% -- 左轴: 三条主曲线 --
yyaxis(ax1, 'left');
p_kp = plot(ax1, time_axis, paths.k_p, style_kp{:}, 'DisplayName', '有效人均私人资本');
p_y  = plot(ax1, time_axis, paths.y,   style_y{:},  'DisplayName', '有效人均产出');
p_w  = plot(ax1, time_axis, paths.w_hat, style_w{:},'DisplayName', '有效工资率');
set(ax1, 'YColor', color_left_axis);
ylim([0,0.8]);
% 由于已经设置了ax1的FontName，Y轴刻度会自动继承Times New Roman。
% 如果有YLabel，显式设置其字体为SimSun
ylabel_left_ax1 = get(ax1, 'YLabel');
if ~isempty(ylabel_left_ax1)
    set(ylabel_left_ax1, 'FontName', 'SimSun');
end


% -- 图例和标签 --
% 标题字体设为SimSun
title_obj_ax1 = title(ax1, '(a) 资本、产出与工资', 'FontSize', font_size_title);
set(title_obj_ax1, 'FontName', 'SimSun');



% 图例：为了确保中文显示，将图例的字体设置为SimSun
legend_obj_ax1 = legend(ax1, [p_kp, p_y, p_w, p_bg_line], 'Location', 'southeast', 'FontSize', 10, 'Box', 'off');
set(legend_obj_ax1, 'FontName', 'SimSun');


% -- 通用设置 --
grid(ax1, 'on');
xlim(ax1, [time_axis(1), time_axis(end)]);
set(ax1, 'XTick', xticks_vec);
xtickangle(ax1, 40);
% X轴刻度（数字和字母）的字体已通过set(ax1, 'FontName', 'Times New Roman')设置。

hold(ax1, 'off');

% --- 面板 2: 年化真实利率 ---
ax2 = nexttile;
hold(ax2, 'on');

% --- 设置坐标轴本身的字体为 Times New Roman ---
set(ax2, 'FontName', 'Times New Roman');
set(ax2, 'FontSize', font_size_axis); % 保持轴字体大小


% -- 右轴: 背景线 --
yyaxis(ax2, 'right');
p_bg_line = plot(ax2, time_axis, paths.retiree_share * 100, style_bg_line{:});
set(ax2, 'YColor', color_right_axis);
ylim(ax2, yy_lim_bar);



% -- 左轴: 利率曲线 --
yyaxis(ax2, 'left');
r_annual = (1 + paths.r).^(1/cS.time_Step) - 1;
plot(ax2, time_axis, r_annual * 100, style_r{:});
set(ax2, 'YColor', color_left_axis);
ylim(ax2, 'padded');


% 图例：为了确保中文显示，将图例的字体设置为SimSun
legend_obj_ax2 = legend(ax2, p_bg_line, ['退休人口占比' newline '(右轴, %)'], ...
    'Location', 'east', 'FontSize', 10, 'Box', 'off', 'FontName', 'SimSun');


% -- 标签 --
% 标题字体设为SimSun
title_obj_ax2 = title(ax2, '(b) 年化真实利率 (%)', 'FontSize', font_size_title, 'FontName', 'SimSun');


% -- 通用设置 --
grid(ax2, 'on');
xlim(ax2, [time_axis(1), time_axis(end)]);
set(ax2, 'XTick', xticks_vec);
xtickangle(ax2, 40);
% X轴刻度（数字和字母）的字体已通过set(ax2, 'FontName', 'Times New Roman')设置。

hold(ax2, 'off');
%% --- 5. 保存图像 ---
fprintf('   正在保存图像至: %s\n', output_filename);
try
    exportgraphics(fig, output_filename, 'Resolution', 600);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');

## 图4.2--固定替代率承诺下的社保基金动态与财政压力 ($\bar{\rho}=40\%$)

In [ ]:
% =========================================================================
% == SCRIPT: exo_pension.m
% ==
% == 版本: [v2.1 - 1x2灰度精修版]
% ==
% == 目的:
% ==   - 采用1x2的横向布局，生成一幅紧凑、专业的灰度图。
% ==   - (a) 左图展示基金存量动态。
% ==   - (b) 右图展示PAYG收支流量动态。
% ==   - 严格遵循特定的尺寸、字体、刻度和颜色要求。
% ==
% == 前置条件:
% ==   - 'TRANS/TPI_results_main_nopps.mat'
% ==   - 'TRANS/iter_results_main_nopps.mat'
% =========================================================================

clear; close all;
addpath(pwd);

fprintf('--- 启动外生养老金基金模拟与绘图脚本 (v2.1) ---\n');

%% --- 1. 用户设定 ---
exo_rho = 0.40;
initial_fund_to_gdp_ratio = 0.055;
year_max = 2100;

%% --- 2. 定义文件路径与加载数据 ---
file_tpi_results = 'TRANS/TPI_results_main_nopps.mat';
file_iter_results = 'TRANS/iter_results_main_nopps.mat';
output_dir = 'tex/fig';
output_filename = fullfile(output_dir, 'fig4.2_exo_pension_fund.png');

if ~exist(file_tpi_results, 'file'), error('找不到TPI结果文件: %s', file_tpi_results); end
if ~exist(file_iter_results, 'file'), error('找不到TPI迭代文件: %s', file_iter_results); end
if ~exist(output_dir, 'dir'), mkdir(output_dir); fprintf('   已创建输出目录: %s\n', output_dir); end

fprintf('   正在加载基准路径数据...\n');
data_tpi = load(file_tpi_results);
data_iter = load(file_iter_results);

cS = data_tpi.cS;
results = data_tpi.results;
b_hat_path_iter = data_iter.b_hat_path_iter;
T = cS.T_sim;

fprintf('   ✅ 数据加载完毕。\n');

%% --- 3. [核心] 会计核算 ---
fprintf('   正在进行养老金缺口与基金动态的会计核算...\n');

r_path_level = results.r_path;
Y_path_level = results.Y_path;
w_hat_path = results.w_path ./ cS.A_path;
mass_retirees_path = sum(cS.Z_path_raw((cS.aR_new+1):end, :), 1);

b_hat_path_exo = exo_rho * w_hat_path;
b_hat_delta = b_hat_path_exo - b_hat_path_iter;

P_level_path = zeros(1, T);
P_level_path(1) = initial_fund_to_gdp_ratio * Y_path_level(1);
fprintf('   初始(t=1)基金余额设定为: %.4f (基于GDP的%.2f%%)\n', P_level_path(1), initial_fund_to_gdp_ratio*100);

for t = 1:(T-1)
    B_delta_total_level = b_hat_delta(t) * cS.A_path(t) * mass_retirees_path(t);
    P_level_path(t+1) = P_level_path(t) * (1 + r_path_level(t)) - B_delta_total_level;
end

Total_PAYG_Contrib_level = b_hat_path_iter .* cS.A_path .* mass_retirees_path;
Total_Promised_Benefit_level = b_hat_path_exo .* cS.A_path .* mass_retirees_path;

contrib_gdp_ratio = Total_PAYG_Contrib_level ./ Y_path_level;
benefit_gdp_ratio = Total_Promised_Benefit_level ./ Y_path_level;

fprintf('   ✅ 会计核算完成。\n');

%% --- 4. 准备绘图数据 ---
fprintf('   正在准备绘图数据...\n');
paths = struct();
paths.fund_level = P_level_path;
paths.retiree_share = mass_retirees_path ./ sum(cS.Z_path_raw, 1);
paths.contrib_gdp = contrib_gdp_ratio;
paths.benefit_gdp = benefit_gdp_ratio;

time_axis_full = (cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step));
time_mask = time_axis_full <= year_max;
time_axis = time_axis_full(time_mask);

fields = fieldnames(paths);
for i = 1:length(fields)
    if size(paths.(fields{i}), 2) == T
        paths.(fields{i}) = paths.(fields{i})(time_mask);
    end
end
fprintf('   图表将展示到 %d 年。\n', year_max);

%% --- 5. [核心] 绘图 ---
fprintf('   正在生成图表...\n');
fig = figure('Name', '外生社保基金动态与流量分析', 'Position', [461, 603, 730, 319]);
tcl = tiledlayout(1, 2, 'Padding', 'compact', 'TileSpacing', 'compact');

% --- 统一定义灰度风格和格式 ---
font_size_title = 12;
font_size_axis  = 10;
color_axis_main = 'black';
color_axis_bg = [0.5 0.5 0.5];



% ================= PANEL (a): 基金存量动态 =================
ax1 = nexttile;
hold(ax1, 'on');

style_fund = {'k-', 'LineWidth', 2};
style_retiree_share = {'--', 'Color', [0.5 0.5 0.5], 'LineWidth', 2};
style_zero_line = {'--', 'Color', [0.7 0.7 0.7], 'LineWidth', 1};

set(ax1, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);

yyaxis(ax1, 'right');
p_retiree = plot(ax1, time_axis, paths.retiree_share * 100, style_retiree_share{:}, 'DisplayName', '退休人口占比 (右轴, %)');
set(ax1, 'YColor', color_axis_bg, 'YLim', [15 70]);

yyaxis(ax1, 'left');
p_fund = plot(ax1, time_axis, paths.fund_level, style_fund{:}, 'DisplayName', '社保基金余额 (左轴)');
ylim([min(paths.fund_level)*0.9 max(paths.fund_level)*2])
yline(ax1, 0, style_zero_line{:});
set(ax1, 'YColor', color_axis_main);
ylabel(ax1, '基金余额 (万亿元)', 'FontName', 'SimSun');


title_obj1 = title(ax1, '(a) 基金存量动态', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend_obj1 = legend(ax1, [p_fund, p_retiree], 'Location', 'south', 'FontSize', 9, 'Box', 'off', 'FontName', 'SimSun');

grid(ax1, 'on'); box(ax1, 'on');
xlim(ax1, [time_axis(1), time_axis(end)]);
set(ax1, 'XTick', time_axis);
xtickangle(ax1, 40);
hold(ax1, 'off');

% ================= PANEL (b): PAYG收支流量 =================
ax2 = nexttile;
hold(ax2, 'on');

style_contrib = {'k-', 'LineWidth', 2};
style_benefit = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2};
color_deficit_area = [0.8 0.8 0.8]; % 浅灰色区域

set(ax2, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);

deficit_gdp = paths.benefit_gdp - paths.contrib_gdp;
p_area = area(ax2, time_axis, deficit_gdp * 100, ...
    'FaceColor', color_deficit_area, 'FaceAlpha', 0.6, ...
    'EdgeColor', 'none', 'DisplayName', 'PAYG缺口');

p_contrib = plot(ax2, time_axis, paths.contrib_gdp * 100, style_contrib{:}, 'DisplayName', 'PAYG缴费收入');
p_benefit = plot(ax2, time_axis, paths.benefit_gdp * 100, style_benefit{:}, 'DisplayName', '承诺的福利支出');

ylim(ax2, [0, max(paths.benefit_gdp)*100*1.1]);
ylabel(ax2, '占GDP的百分比 (%)', 'FontName', 'SimSun');

title_obj2 = title(ax2, '(b) PAYG收支流量', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend_obj2 = legend(ax2, [p_contrib, p_benefit, p_area], 'Location', 'northwest', 'FontSize', 9, 'Box', 'off', 'FontName', 'SimSun');

grid(ax2, 'on'); box(ax2, 'on');
xlim(ax2, [time_axis(1), time_axis(end)]);
set(ax2, 'XTick', time_axis);
xtickangle(ax2, 40);
hold(ax2, 'off');


fprintf('   ✅ 图表生成完毕。\n');

%% --- 6. 保存图像 ---
fprintf('   正在保存图像至: %s\n', output_filename);
try
    exportgraphics(fig, output_filename, 'Resolution', 600);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');